https://www.tensorflow.org/tutorials/keras/keras_tuner#define_the_model

In [1]:
import tensorflow as tf
from tensorflow import keras

import keras_tuner as kt

In [2]:
(img_train, label_train), (img_test, label_test) = keras.datasets.fashion_mnist.load_data()

In [3]:
# Normalize pixel values between 0 and 1
img_train = img_train.astype('float32') / 255.0
img_test = img_test.astype('float32') / 255.0

In [4]:
def model_builder(hp):
    model = keras.Sequential()
    model.add(keras.layers.Flatten(input_shape=(28, 28)))

    # Tune the number of units in the first Dense layer. Choose an optimal value between 32-512
    hp_units = hp.Int('units', min_value=32, max_value=512, step=32)

    model.add(keras.layers.Dense(units=hp_units, activation='relu'))
    model.add(keras.layers.Dense(10))

    # Tune the learning rate for the optimizer. Choose an optimal value from 0.01, 0.001, or 0.0001
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
        loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=['accuracy']
    )

    return model

In [5]:
tuner = kt.Hyperband(
    model_builder,
    objective='val_accuracy',
    max_epochs=10,
    factor=3,
    directory='tuners',
    project_name='hypertuner'
)

In [6]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

In [7]:
tuner.search(
    img_train,
    label_train,
    epochs=50,
    validation_split=0.2,
    callbacks=[stop_early]
)

# Get the optimal hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

print(f'''
The hyperparameter search is complete. The optimal number of units in the first densely-connected layer is {best_hps.get('units')} and the optimal learning rate for the optimizer is {best_hps.get('learning_rate')}
''')

Trial 30 Complete [00h 00m 38s]
val_accuracy: 0.8841666579246521

Best val_accuracy So Far: 0.8944166898727417
Total elapsed time: 00h 07m 48s
INFO:tensorflow:Oracle triggered exit

The hyperparameter search is complete. The optimal number of units in the first densely-connected layer is 512 and the optimal learning rate for the optimizer is 0.001



In [8]:
# Build the model with the optimal hyperparameters and train it on the data for 50 epochs
model = tuner.hypermodel.build(best_hps)

history = model.fit(
    img_train,
    label_train,
    epochs=50,
    validation_split=0.2
)

val_acc_per_epoch = history.history['val_accuracy']

best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
print(f'Best epoch: {best_epoch}')

Epoch 1/50
1500/1500 [==============================] - 4s 3ms/step - loss: 0.4906 - accuracy: 0.8261 - val_loss: 0.4191 - val_accuracy: 0.8432
Epoch 2/50
1500/1500 [==============================] - 4s 2ms/step - loss: 0.3698 - accuracy: 0.8660 - val_loss: 0.3473 - val_accuracy: 0.8728
Epoch 3/50
1500/1500 [==============================] - 4s 2ms/step - loss: 0.3280 - accuracy: 0.8784 - val_loss: 0.3455 - val_accuracy: 0.8748
Epoch 4/50
1500/1500 [==============================] - 4s 2ms/step - loss: 0.3056 - accuracy: 0.8872 - val_loss: 0.3411 - val_accuracy: 0.8767
Epoch 5/50
1500/1500 [==============================] - 4s 2ms/step - loss: 0.2848 - accuracy: 0.8952 - val_loss: 0.3299 - val_accuracy: 0.8822
Epoch 6/50
1500/1500 [==============================] - 4s 2ms/step - loss: 0.2683 - accuracy: 0.9010 - val_loss: 0.3213 - val_accuracy: 0.8866
Epoch 7/50
1500/1500 [==============================] - 4s 2ms/step - loss: 0.2570 - accuracy: 0.9048 - val_loss: 0.3225 - val_accuracy:

In [9]:
hypermodel = tuner.hypermodel.build(best_hps)

# Retrain the model
hypermodel.fit(
    img_train,
    label_train,
    epochs=best_epoch,
    validation_split=0.2
)

Epoch 1/36
1500/1500 [==============================] - 4s 3ms/step - loss: 0.4931 - accuracy: 0.8245 - val_loss: 0.3904 - val_accuracy: 0.8585
Epoch 2/36
1500/1500 [==============================] - 4s 2ms/step - loss: 0.3713 - accuracy: 0.8651 - val_loss: 0.4065 - val_accuracy: 0.8522
Epoch 3/36
1500/1500 [==============================] - 4s 2ms/step - loss: 0.3295 - accuracy: 0.8792 - val_loss: 0.3528 - val_accuracy: 0.8673
Epoch 4/36
1500/1500 [==============================] - 4s 2ms/step - loss: 0.3041 - accuracy: 0.8887 - val_loss: 0.3381 - val_accuracy: 0.8788
Epoch 5/36
1500/1500 [==============================] - 4s 2ms/step - loss: 0.2848 - accuracy: 0.8934 - val_loss: 0.3220 - val_accuracy: 0.8847
Epoch 6/36
1500/1500 [==============================] - 4s 2ms/step - loss: 0.2701 - accuracy: 0.8997 - val_loss: 0.3361 - val_accuracy: 0.8828
Epoch 7/36
1500/1500 [==============================] - 4s 2ms/step - loss: 0.2531 - accuracy: 0.9046 - val_loss: 0.3332 - val_accuracy:

In [11]:
eval_result = hypermodel.evaluate(img_test, label_test)
print(f'[Test loss, Test accuracy]: {eval_result}')

313/313 [==============================] - 1s 3ms/step - loss: 0.5137 - accuracy: 0.8882
[Test loss, Test accuracy]: [0.5136528611183167, 0.8881999850273132]
